In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.use('Cairo')  # for saving SVGs that Affinity Designer can parse
import matplotlib.pyplot as plt
import seaborn as sns
import cloudpickle as pkl

import candas as can
import gumbi as gmb
from candas.test import FluorescenceData, QuantStudio

from sklearn.metrics import confusion_matrix
from scipy.integrate import trapz

import benchmarking as bench

base_pth, code_pth, data_pth, _, fig_pth = can.utils.setup_paths(make_missing=False)
plt.style.use('style.mplstyle')

%config InlineBackend.figure_format = 'retina'

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


# Experimental Results

In [3]:
endpoints = pd.read_csv(data_pth / 'JG073N Endpoints.csv').rename(columns={'SignalDifference': 'FAM-HEX'})

ng1_endpoints = endpoints[
    (endpoints["IFI44L Copies"] > 0) &
    (endpoints["EMRI Copies"] > 0) &
    (endpoints["Material"] == "RNA") &
    (endpoints["ng RNA"] == 1)
]

patients_100ng = pd.read_csv(data_pth / 'JG073 Processed patient data.csv')
patients_1ng = (
    patients_100ng
    .drop(columns=['FAM89A'])
    .assign(
        EMRI = patients_100ng.EMRI - 2,
        IFI44L = patients_100ng.IFI44L - 2
        )
    )


In [4]:
ng1_ds = gmb.DataSet(ng1_endpoints, outputs=['FAM-HEX'])
ng1_gp = gmb.GP(ng1_ds).fit(
    continuous_dims = ['EMRI Copies', 'IFI44L Copies'],
    linear_dims = ['EMRI Copies', 'IFI44L Copies']
)

In [5]:
import importlib as il
il.reload(bench)

obj = bench.Objective(ng1_gp, ng1_ds, patients_1ng, 2, data_pth / "JG073 Logistic Regression Scores.csv")
obj.set_limits(pad=0)
obj.predict()
obj.optimize_scale_shift(show=False)
obj.set_limits(pad=0.1)
obj.predict()
obj.score_patients()
obj.build_roc();

In [6]:
LR_roc = []
scores = obj.LR_Scores["LR_Score_m"]
dxs = obj.LR_Scores["Diagnosis"]
thresholds = np.hstack(
    [scores.min() - 1, sorted(scores.values), scores.max() + 1]
)
for thresh in thresholds:
    tn, fp, fn, tp = confusion_matrix(dxs == "Viral", scores <= thresh).ravel()
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)
    LR_roc.append([tpr, fpr])
    
LR_auroc = trapz(np.array(LR_roc)[:, 0], np.array(LR_roc)[:, 1])

In [7]:
width = 1.725
height = 1.75
figsize = (width, height)
ticklabelsize = 6
labelsize = 6

mar_l = 0.4
mar_r = 0.05
mar_t = 0.05
mar_b = 0.35

def format_fig(fig, figsize=figsize, mar_l=mar_l, mar_r=mar_r, mar_t=mar_t, mar_b=mar_b, **kwargs):
    """Adjust margins of subplots using figsize"""
    height, width = figsize
    fig.set_size_inches(figsize)
    
    plt.subplots_adjust(
        left=mar_l / width,
        right=1 - mar_r / width,
        top=1 - mar_t / height,
        bottom=mar_b / height,
        **kwargs
    )
    
    for ax in fig.get_axes():
        ax.tick_params(which='both', length=1.5, width=0.6, labelsize=8)

In [14]:
fig, ax = plt.subplots(1, 1, figsize=figsize)
obj.set_limits(pad=0.1)
obj.predict()
pp = gmb.ParrayPlotter(x=obj.IFI44L_grid, y=obj.EMRI_grid, z=(obj.sig + 3*obj.vshift) / obj.scale)
pp(plt.contourf, cmap=obj.cmap, norm=mpl.colors.CenteredNorm(), zorder=-10)
cbar = pp.colorbar(ax=ax)#, ticks=[-1, -0.5, 0, 0.5, 1])
pp(plt.contour, levels=0, colors="k", norm=mpl.colors.CenteredNorm(), zorder=-5)
cbar.set_label("FAM-HEX", fontsize=8)
cbar.set_label("")
cbar.ax.tick_params(labelsize=8)
obj.plot_observations(ax=ax, s=6**2)
obj.plot_patients(ax=ax, s=4**2)
ax.set_xticks([1,2,3,4,5,6])
format_fig(fig, mar_l=0.325, mar_r=0.075)
plt.savefig(fig_pth / 'JG073N predictions.png', dpi=600, transparent=True);
plt.savefig(fig_pth / 'JG073N predictions.svg', dpi=600, transparent=True);

fig, ax = plt.subplots(1, 1, figsize=figsize)
LR_color = '#2F193B'
CAN_color = '#A02A59'
# Plot LR_roc
tpr_values, fpr_values = zip(*LR_roc)
ax.plot(fpr_values, tpr_values, color = LR_color)
ax.set_ylabel("Sensitivity")
ax.set_xlabel("1-Specificity")
format_fig(fig, mar_l=0.5)

obj.plot_roc(ax=ax, color=CAN_color)
obj.roc['m'] = np.array(obj.roc['m'])
xl, yl = ax.get_xlim(), ax.get_ylim()

xy=(np.diff(xl)*0.95+xl[0], np.diff(yl)*0.25+yl[0])
ax.annotate('AUROC', xy=xy, ha='right', color = 'k')

xy=(np.diff(xl)*0.95+xl[0], np.diff(yl)*0.15+yl[0])
ax.annotate(f'LR: {LR_auroc:0.3f}', xy=xy, ha='right', color = LR_color)

xy=(np.diff(xl)*0.95+xl[0], np.diff(yl)*0.05+yl[0])
ax.annotate(f'Assay: {obj.auroc("m"):0.3f}', xy=xy, ha='right', color=CAN_color)
plt.savefig(fig_pth / 'JG073N ROC.png', dpi=600, transparent=True);
plt.savefig(fig_pth / 'JG073N ROC.svg', dpi=600, transparent=True);

fig, ax = plt.subplots(1, 1, figsize=figsize)
obj.plot_comparison(ax=ax, scatter_kws={'s': 6**2}, line_kws={'alpha':0.25, 'zorder':-1})
xl, yl = ax.get_xlim(), ax.get_ylim()
xy=(np.diff(xl)*0.95+xl[0], np.diff(yl)*0.05+yl[0])
ax.annotate(f'$R^2$: {obj.reg.rvalue**2:0.3f}', xy=xy, ha='right')
format_fig(fig)
plt.savefig(fig_pth / 'JG073N comparison.png', dpi=600, transparent=True);
plt.savefig(fig_pth / 'JG073N comparison.svg', dpi=600, transparent=True);

# Logistic Regression

In [9]:
LR_rslt_file = data_pth / 'JG073 Bayesian Logistic Regression Results.pkl'

with open(LR_rslt_file,'rb') as buff:
    LR_results = pkl.load(buff)

lg10_Copies = LR_results['data']
summary = LR_results['summary']
per_gene_posterior_predictive = LR_results['per_gene_posterior_predictive']
per_gene_posterior_summary_stats = LR_results['per_gene_posterior_summary_stats']
per_gene_lines = LR_results['per_gene_lines']

genes = list(per_gene_posterior_predictive.keys())

In [10]:
from scipy.special import logit, expit

In [11]:
logit_max=5
space = 'log odds'
figsize = (18, 4)
n_c = 100
copies = np.linspace(3, 7, n_c)

mar_l = 0.475
mar_r = 0.05
mar_t = 0.05
mar_b = 0.45

ymax = logit_max
yax1_ticks = np.arange(-(ymax), (ymax)+0.1, 2.5)  # log-odds
yax1_ticks = [-4, -2, 0, 2, 4]  # log-odds
yax2_pts = np.array([0.0001, 0.001, 0.01, 0.1, 0.5, 0.9, 0.99, 0.999, 0.9999])  # probabilities
yax2_ticks = logit(yax2_pts)
ylim = [-ymax, ymax]
yax1_label = ''  # 'Conditional\nLog Odds Dx'
yax2_label = ''  # 'Conditional\nProbability of Dx'
hline = 0

ax2s = []

fig, axs = plt.subplots(2, 1, figsize=figsize, sharex=True, sharey=True)
for i,g in enumerate(genes):
    ax = axs[i]
    ax.axhline(hline, color = 'k', zorder=0)

    plt.setp(ax,
                # title  = g,
                yticks = yax1_ticks,
                ylim   = ylim,
                xticks = np.arange(copies.min(), copies.max()+1),
                xlim   = [copies[0]-(copies[-1]-copies[0])*0.05,copies[-1]+(copies[-1]-copies[0])*0.05],
            );
    ax.set_title(g, fontname='BreveSansText-Light', fontsize=10, pad=2)

    ax.set_ylabel(yax1_label)

    # For clarity, include a second yaxis that shows the marginal probability rather than log-odds
    ax2 = ax.twinx()
    ax2s.append(ax2)
    plt.setp(ax2,
                yticks = yax2_ticks,
                yticklabels = yax2_pts,
            )
    ax2.set_ylim(ax.get_ylim())
    ax2.set_ylabel(yax2_label,
                    rotation = -90,
                    verticalalignment='bottom')
    
# Plot patient data
for i,g in enumerate(genes):
    ax = axs[i]
    x = lg10_Copies[g]
    y = (lg10_Copies.Bacterial*2-1)*(ymax-0.5)
        
    ax.plot(x, y, ls='none', marker='o', mfc='k', mec='none', ms=4, alpha=0.2)
    
for i,g in enumerate(genes):
    post = per_gene_posterior_predictive[g]
        
    lim = (copies>lg10_Copies[g].min()) & (copies<lg10_Copies[g].max())
        
    l,m,u = [np.percentile(post, p, axis=1) for p in [2.5, 50, 97.5]]
    axs[i].fill_between(copies[lim], l[lim], u[lim], alpha=0.2, facecolor=f'C{i}')
    axs[i].plot(copies[lim], m[lim], color=f'C{i}')
    axs[i].plot(copies[lim], post[lim,:30], alpha=0.1, color=f'C{i}')
    
axs[-1].set_xlabel('log$_{10}$ Copies', labelpad=0)

fig.text(0.06, 0.55, 'Log-Odds Bacterial', ha='center', va='center', rotation='vertical')
fig.text(0.975, 0.55, 'Probability Bacterial', ha='right', va='center', rotation=270)
    
# plt.tight_layout()
format_fig(fig, mar_l=0.35, mar_r=0.5, mar_t=0.2, hspace=0.5)
plt.savefig(fig_pth / 'JG073N patient data.png', dpi=600, transparent=True);